In [11]:
import sys
sys.path.append('../build/')
%pylab inline
np.set_printoptions(precision=4, suppress=True)
import pandas as pd
import linear_solver
import versor as vsr

Populating the interactive namespace from numpy and matplotlib


In [12]:
# import autograd.numpy as np

from pymanopt.manifolds import manifold
from pymanopt.manifolds import Sphere
from pymanopt import Problem
from pymanopt.solvers import SteepestDescent, TrustRegions, ParticleSwarm, ConjugateGradient, NelderMead
from __future__ import division

import numpy as np
import numpy.linalg as la
import numpy.random as rnd

from pymanopt.manifolds.manifold import Manifold

In [13]:
def create_rotor(th_lims=(0, np.pi/2)):
    rotor = vsr.Rot(vsr.Biv(*np.random.uniform(-1, 1, 3)).unit()
                      * np.random.uniform(*th_lims) * -0.5)
    return rotor

In [14]:
class Motor(Manifold):
    def __init__(self):
        self._shape = 8
        self._name = "Motor manifold"

    def __str__(self):
        return self._name
    
    def norm(self, X, U):
        return la.norm(U)
    
    def retr(self, X, U):
        Y = vsr.Mot(*X) + vsr.Mot(*U)
        return np.array(Y.retract())
    
    def CayleySelig(self, X, U):
        M = vsr.Mot(*X)
        B = M.rev() * vsr.Mot(*U)
        
        Rp = vsr.Mot(1.0, B[1], B[2], B[3], 0.0, 0.0, 0.0, 0.0)
        Rn = vsr.Mot(1.0, -B[1], -B[2], -B[3], 0.0, 0.0, 0.0, 0.0)
        Rninv = Rn.inv()
        eps = vsr.Mot(0,0,0,0,0,0,0,-1)
        b = vsr.Mot(0.0, B[6], -B[5], B[4], 0.0, 0.0, 0.0, 0.0)
        return np.array(M * (Rp * Rninv + eps * Rninv * b * Rninv * 2))

    def oexp(self,X,U):

        M = vsr.Mot(*X)
        B = M.rev() * vsr.Mot(*U)
        n = np.sqrt(1 + B[1] * B[1] + B[2] * B[2] + B[3] * B[3])
        s = B[1] * B[6] - B[2] * B[5] + B[3] * B[4]
        m = vsr.Mot(1.0, B[1], B[2], B[3], B[4], B[5], B[6], s) * (1.0 / n)
        return np.array(M * m)

    def CayleyLi(self,X, U):        
        M = vsr.Mot(*X)
        B = M.rev() * vsr.Mot(*U)
        B_ = vsr.Mot(0.0, B[1], B[2], B[3], B[4], B[5], B[6], 0.0)
        BB = B_ * B_
        Rp = vsr.Mot(1.0, B[1], B[2], B[3], B[4], B[5], B[6], 0.0)
        R0 = vsr.Mot(1.0 - BB[0], 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
        R4 = vsr.Mot(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, BB[7])
        Rn = R0 + R4
        Rden = R0 * R0 
        return np.array(M * (Rp * Rp * Rn * Rden.inv()))

    def exp(self, X, U):
        M = vsr.Mot(*X)
        B = M.rev() * vsr.Mot(*U)
        return np.array(M * vsr.Dll(B[1], B[2], B[3], B[4], B[5], B[6]).exp())
    
#     retr = CayleySelig
#     retr = oexp
#     retr = CayleyLi
    retr = exp

    @property
    def dim(self):
        return 6

In [15]:
class Rotator(Sphere):
    def __init__(self):
        super().__init__(4)
        
    def exp(self, X, U):
        R = vsr.Rot(*X)
        g = R.rev() * vsr.Rot(*U)
        B = vsr.Biv(g[1], g[2], g[3]) * -1.0
        return np.array(R * B.exp())
    
    def cay(self, X, U):
        R = vsr.Rot(*X)
        g = R.rev() * vsr.Rot(*U)
        Y = vsr.Rot(*self._normalize(np.array([1.0, g[1], g[2], g[3]])))
        return np.array(R * Y)
    
    retr = cay
        
        

In [16]:
def create_motor(d_lims=(0, 1), th_lims=(0, np.pi)):
    translator = (vsr.Vec(*np.random.random(3)).unit()
                  * np.random.uniform(*d_lims)).trs()
    rotator = vsr.Rot(vsr.Biv(*np.random.uniform(-1, 1, 3)).unit()
                      * np.random.uniform(*th_lims) * -0.5)
    motor = translator * rotator
    return motor

In [395]:
n_points=10

m0 = create_motor(th_lims=(0,np.pi/3))
points_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)).null() for i in range(n_points)]
points_b = [point.spin(m0) for point in points_a]

In [396]:
def grade2(Y, M):
    YM2 = Y * M
#     print(YM2)
    YM2[0] = 0.0
    YM2[26] = 0.0
    YM2[27] = 0.0
    YM2[28] = 0.0
    YM2[29] = 0.0
    YM2[30] = 0.0
    return YM2

In [397]:
M = vsr.CGA(create_motor())

In [398]:
def proj(X):
    M = vsr.Mot(0,0,0,0,0,0,0,0)
    sgns = [1,2,3,7]
    for i in range(8):
        Ei = vsr.MotRec(0,0,0,0,0,0,0,0)
        if i in sgns:
            Ei[i] = -1.0
        else:
            Ei[i] = 1.0
            
        Ei[i] = 1.0
        M[i] = (X * vsr.CGA(Ei))[0]
    return vsr.CGA(M)

In [399]:
def adproj(X):
    M = vsr.MotRec(0,0,0,0,0,0,0,0)
    sgns = [1,2,3,7]
    for i in range(8):
        Ei = vsr.Mot(0,0,0,0,0,0,0,0)
        if i in sgns:
            Ei[i] = -1.0
        else:
            Ei[i] = 1.0
            
        M[i] = (vsr.CGA(Ei) * X)[0]
    return vsr.CGA(M)

In [400]:
A = vsr.CGA(points_a[0])
B = vsr.CGA(points_b[0])
Y = grade2(A * M.rev() * B + A.rev() * M.rev() * B.rev(), M)
Y

CGA: [ 0 0 0 0 0 0 -0.31 -0.22 -0.45 0.16 0.61 0.2 0.16 0.24 -0.068 0.2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]

In [401]:
proj(Y)

CGA: [ 0 0 0 0 0 0 0.31 0.22 0.45 0 0 0 0.16 0.24 -0.068 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]

In [411]:
adproj(Y * M.rev())

CGA: [ 0.17 0 0 0 0 0 -0.39 -0.2 -0.38 0.14 0.62 0.19 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -0.012 0 0 0 0 0 ]

In [406]:
(M.rev() * vsr.CGA(vsr.Mot(*gradk3(points_a[0], points_b[0], np.array(vsr.Mot(M)))))) * M.rev()

CGA: [ 0.0016 0 0 0 0 0 -0.32 -0.2 -0.45 0 0 0 -0.23 -0.65 -0.08 0 0 0 0 0 0 0 0 0 0 0 0 -0.07 0 0 0 0 ]

In [380]:
def rotor_rotate_vector(a, X):
    a1, a2, a3 = np.array(a)
    r0, r1, r2, r3 = np.array(X)
    return (a1 * r3 * r3 + (2.0 * a3 * r1 - 2.0 * a2 * r2) * r3 - a1 * r2 * r2 + 2.0 * a3 * r0 * r2 - a1 * r1 * r1 + 2.0 * a2 * r0 * r1 + a1 * r0 * r0,
            (-a2) * r3 * r3 + (2.0 * a3 * r0 - 2.0 * a1 * r2) * r3 + a2 * r2 * r2 - 2.0 * a3 * r1 * r2 - a2 * r1 * r1 - 2.0 * a1 * r0 * r1 + a2 * r0 * r0,
            (-a3) * r3 * r3 + (2.0 * a1 * r1 - 2.0 * a2 * r0) * r3 - a3 * r2 * r2 + (-2.0 * a2 * r1 - 2.0 * a1 * r0) * r2 + a3 * r1 * r1 + a3 * r0 * r0)

In [19]:
def gradk(a, b, X):
    a1, a2, a3 = np.array(a)
    b1, b2, b3 = np.array(b)
    r0, r1, r2, r3 = np.array(X)
    return np.array(vsr.Rot(r0, r1, r2, r3) * vsr.Biv(
        (-2.0 * a1 * b2 - 2.0 * a2 * b1) * r3 * r3 + ((4.0 * a2 * b2 - 4.0 * a1 * b1) * r2 - 4.0 * a2 * b3 * r1 - 4.0 * a1 * b3 * r0) * r3 + (2.0 * a1 * b2 + 2.0 * a2 * b1) * r2 * r2 + (4.0 * a2 * b3 * r0 - 4.0 * a1 * b3 * r1) * r2 + (2.0 * a2 * b1 - 2.0 * a1 * b2) * r1 * r1 + (4.0 * a2 * b2 + 4.0 * a1 * b1) * r0 * r1 + (2.0 * a1 * b2 - 2.0 * a2 * b1) * r0 * r0,
        (-2.0 * a1 * b3 - 2.0 * a3 * b1) * r3 * r3 + (4.0 * a3 * b2 * r2 + (4.0 * a1 * b1 - 4.0 * a3 * b3) * r1 + 4.0 * a1 * b2 * r0) * r3 + (2.0 * a3 * b1 - 2.0 * a1 * b3) * r2 * r2 + ((4.0 * a3 * b3 + 4.0 * a1 * b1) * r0 - 4.0 * a1 * b2 * r1) * r2 + (2.0 * a1 * b3 + 2.0 * a3 * b1) * r1 * r1 + 4.0 * a3 * b2 * r0 * r1 + (2.0 * a1 * b3 - 2.0 * a3 * b1) * r0 * r0,
        (2.0 * a3 * b2 - 2.0 * a2 * b3) * r3 * r3 + (4.0 * a3 * b1 * r2 + 4.0 * a2 * b1 * r1 + (4.0 * a3 * b3 + 4.0 * a2 * b2) * r0) * r3 + (-2.0 * a2 * b3 - 2.0 * a3 * b2) * r2 * r2 + ((4.0 * a3 * b3 - 4.0 * a2 * b2) * r1 + 4.0 * a2 * b1 * r0) * r2 + (2.0 * a2 * b3 + 2.0 * a3 * b2) * r1 * r1 - 4.0 * a3 * b1 * r0 * r1 + (2.0 * a2 * b3 - 2.0 * a3 * b2) * r0 * r0))

In [20]:
def grad_explicit(X): 
    g = np.sum([gradk(a, b, X) for a, b in zip(points_a, points_b)], axis=0) * 2.0
    return g

In [21]:
def costk3(a, b, X):
    a1, a2, a3, a4, a5 = np.array(a)
    b1, b2, b3, b4, b5 = np.array(b)
    m1, m2, m3, m4, m5, m6, m7, m8 = X
    return 4.0 * a4 * b4 * m8 * m8 + ((4.0 * a4 * b1 - 4.0 * a1 * b4) * m4 + (4.0 * a2 * b4 - 4.0 * a4 * b2) * m3 + (4.0 * a4 * b3 - 4.0 * a3 * b4) * m2) * m8 + 4.0 * a4 * b4 * m7 * m7 + ((4.0 * a2 * b4 + 4.0 * a4 * b2) * m4 + (4.0 * a1 * b4 + 4.0 * a4 * b1) * m3 + (4.0 * a4 * b3 - 4.0 * a3 * b4) * m1) * m7 + 4.0 * a4 * b4 * m6 * m6 + ((-4.0 * a3 * b4 - 4.0 * a4 * b3) * m4 + (4.0 * a1 * b4 + 4.0 * a4 * b1) * m2 + (4.0 * a4 * b2 - 4.0 * a2 * b4) * m1) * m6 + 4.0 * a4 * b4 * m5 * m5 + ((-4.0 * a3 * b4 - 4.0 * a4 * b3) * m3 + (-4.0 * a2 * b4 - 4.0 * a4 * b2) * m2 + (4.0 * a4 * b1 - 4.0 * a1 * b4) * m1) * m5 + (2.0 * a4 * b5 + 2.0 * a5 * b4 + 2.0 * a3 * b3 + 2.0 * a2 * b2 - 2.0 * a1 * b1) * m4 * m4 + ((4.0 * a1 * b2 + 4.0 * a2 * b1) * m3 + (-4.0 * a1 * b3 - 4.0 * a3 * b1) * m2 + (4.0 * a2 * b3 - 4.0 * a3 * b2) * m1) * m4 + (2.0 * a4 * b5 + 2.0 * a5 * b4 + 2.0 * a3 * b3 - 2.0 * a2 * b2 + 2.0 * a1 * b1) * m3 * m3 + ((4.0 * a2 * b3 + 4.0 * a3 * b2) * m2 + (4.0 * a1 * b3 - 4.0 * a3 * b1) * m1) * m3 + (2.0 * a4 * b5 + 2.0 * a5 * b4 - 2.0 * a3 * b3 + 2.0 * a2 * b2 + 2.0 * a1 * b1) * m2 * m2 + (4.0 * a1 * b2 - 4.0 * a2 * b1) * m1 * m2 + (2.0 * a4 * b5 + 2.0 * a5 * b4 - 2.0 * a3 * b3 - 2.0 * a2 * b2 - 2.0 * a1 * b1) * m1 * m1

In [22]:
def cost3(X): 
    return np.sum([costk3(a, b, X) for a, b in zip(points_a, points_b)])

In [23]:
def gradk3(a,b,X):
    a1, a2, a3, a4, a5 = np.array(a)
    b1, b2, b3, b4, b5 = np.array(b)
    m1, m2, m3, m4, m5, m6, m7, m8 = X
    B0 = (-4.0 * a2 * b4 * m4 - 4.0 * a1 * b4 * m3) * m8 + (4.0 * a2 * b4 * m3 - 4.0 * a1 * b4 * m4) * m7 + (4.0 * a2 * b4 * m2 + 4.0 * a1 * b4 * m1) * m6 + (4.0 * a1 * b4 * m2 - 4.0 * a2 * b4 * m1) * m5 + (-2.0 * a1 * b2 - 2.0 * a2 * b1) * m4 * m4 + ((4.0 * a2 * b2 - 4.0 * a1 * b1) * m3 - 4.0 * a2 * b3 * m2 - 4.0 * a1 * b3 * m1) * m4 + (2.0 * a1 * b2 + 2.0 * a2 * b1) * m3 * m3 + (4.0 * a2 * b3 * m1 - 4.0 * a1 * b3 * m2) * m3 + (2.0 * a2 * b1 - 2.0 * a1 * b2) * m2 * m2 + (4.0 * a2 * b2 + 4.0 * a1 * b1) * m1 * m2 + (2.0 * a1 * b2 - 2.0 * a2 * b1) * m1 * m1
    B1 = (4.0 * a1 * b4 * m2 - 4.0 * a3 * b4 * m4) * m8 + (4.0 * a3 * b4 * m3 + 4.0 * a1 * b4 * m1) * m7 + (4.0 * a1 * b4 * m4 + 4.0 * a3 * b4 * m2) * m6 + (4.0 * a1 * b4 * m3 - 4.0 * a3 * b4 * m1) * m5 + (-2.0 * a1 * b3 - 2.0 * a3 * b1) * m4 * m4 + (4.0 * a3 * b2 * m3 + (4.0 * a1 * b1 - 4.0 * a3 * b3) * m2 + 4.0 * a1 * b2 * m1) * m4 + (2.0 * a3 * b1 - 2.0 * a1 * b3) * m3 * m3 + ((4.0 * a3 * b3 + 4.0 * a1 * b1) * m1 - 4.0 * a1 * b2 * m2) * m3 + (2.0 * a1 * b3 + 2.0 * a3 * b1) * m2 * m2 + 4.0 * a3 * b2 * m1 * m2 + (2.0 * a1 * b3 - 2.0 * a3 * b1) * m1 * m1
    B2 = (4.0 * a3 * b4 * m3 + 4.0 * a2 * b4 * m2) * m8 + (4.0 * a3 * b4 * m4 + 4.0 * a2 * b4 * m1) * m7 + (4.0 * a2 * b4 * m4 - 4.0 * a3 * b4 * m1) * m6 + (4.0 * a2 * b4 * m3 - 4.0 * a3 * b4 * m2) * m5 + (2.0 * a3 * b2 - 2.0 * a2 * b3) * m4 * m4 + (4.0 * a3 * b1 * m3 + 4.0 * a2 * b1 * m2 + (4.0 * a3 * b3 + 4.0 * a2 * b2) * m1) * m4 + (-2.0 * a2 * b3 - 2.0 * a3 * b2) * m3 * m3 + ((4.0 * a3 * b3 - 4.0 * a2 * b2) * m2 + 4.0 * a2 * b1 * m1) * m3 + (2.0 * a2 * b3 + 2.0 * a3 * b2) * m2 * m2 - 4.0 * a3 * b1 * m1 * m2 + (2.0 * a2 * b3 - 2.0 * a3 * b2) * m1 * m1
    B3 = 4.0 * a4 * b4 * m4 * m8 - 4.0 * a4 * b4 * m3 * m7 - 4.0 * a4 * b4 * m2 * m6 + 4.0 * a4 * b4 * m1 * m5 + (2.0 * a4 * b1 - 2.0 * a1 * b4) * m4 * m4 + (4.0 * a4 * b3 * m2 - 4.0 * a4 * b2 * m3) * m4 + (-2.0 * a1 * b4 - 2.0 * a4 * b1) * m3 * m3 - 4.0 * a4 * b3 * m1 * m3 + (-2.0 * a1 * b4 - 2.0 * a4 * b1) * m2 * m2 - 4.0 * a4 * b2 * m1 * m2 + (2.0 * a4 * b1 - 2.0 * a1 * b4) * m1 * m1
    B4 = -4.0 * a4 * b4 * m3 * m8 - 4.0 * a4 * b4 * m4 * m7 + 4.0 * a4 * b4 * m1 * m6 + 4.0 * a4 * b4 * m2 * m5 + (-2.0 * a2 * b4 - 2.0 * a4 * b2) * m4 * m4 + (-4.0 * a4 * b1 * m3 - 4.0 * a4 * b3 * m1) * m4 + (2.0 * a4 * b2 - 2.0 * a2 * b4) * m3 * m3 - 4.0 * a4 * b3 * m2 * m3 + (-2.0 * a2 * b4 - 2.0 * a4 * b2) * m2 * m2 + 4.0 * a4 * b1 * m1 * m2 + (2.0 * a4 * b2 - 2.0 * a2 * b4) * m1 * m1
    B5 = 4.0 * a4 * b4 * m2 * m8 + 4.0 * a4 * b4 * m1 * m7 + 4.0 * a4 * b4 * m4 * m6 + 4.0 * a4 * b4 * m3 * m5 + (-2.0 * a3 * b4 - 2.0 * a4 * b3) * m4 * m4 + (4.0 * a4 * b1 * m2 + 4.0 * a4 * b2 * m1) * m4 + (-2.0 * a3 * b4 - 2.0 * a4 * b3) * m3 * m3 + (4.0 * a4 * b1 * m1 - 4.0 * a4 * b2 * m2) * m3 + (2.0 * a4 * b3 - 2.0 * a3 * b4) * m2 * m2 + (2.0 * a4 * b3 - 2.0 * a3 * b4) * m1 * m1
    return np.array(vsr.Mot(m1, m2, m3, m4, m5, m6, m7, m8) * vsr.Dll(B0, B1, B2, B3, B4, B5))

In [24]:
def grad_explicit3(X): 
    g = np.sum([gradk3(a, b, X) for a, b in zip(points_a, points_b)], axis=0) * 2.0
    return g

In [66]:
def costk(a, b, X): 
    a1, a2, a3 = np.array(a)
    b1, b2, b3 = np.array(b)
    r0, r1, r2, r3 = X
    o0 = a1 * r3 * r3 + (2.0 * a3 * r1 - 2.0 * a2 * r2) * r3 - a1 * r2 * r2 + 2.0 * a3 * r0 * r2 - a1 * r1 * r1 + 2.0 * a2 * r0 * r1 + a1 * r0 * r0 - b1
    o1 = (-a2) * r3 * r3 + (2.0 * a3 * r0 - 2.0 * a1 * r2) * r3 + a2 * r2 * r2 - 2.0 * a3 * r1 * r2 - a2 * r1 * r1 - 2.0 * a1 * r0 * r1 + a2 * r0 * r0 - b2
    o2 = (-a3) * r3 * r3 + (2.0 * a1 * r1 - 2.0 * a2 * r0) * r3 - a3 * r2 * r2 + (-2.0 * a2 * r1 - 2.0 * a1 * r0) * r2 + a3 * r1 * r1 + a3 * r0 * r0 - b3
    return o0 * o0 + o1 * o1 + o2 * o2

def costk2(a, b, X):
    a1, a2, a3 = np.array(a)
    b1, b2, b3 = np.array(b)
    r0, r1, r2, r3 = X
    return ((a3 * b3 + a2 * b2 - a1 * b1) * r3 * r3 +
            ((2.0 * a1 * b2 + 2.0 * a2 * b1) * r2 + 
             (-2.0 * a1 * b3 - 2.0 * a3 * b1) * r1 + 
             (2.0 * a2 * b3 - 2.0 * a3 * b2) * r0) * r3 + 
            (a3 * b3 - a2 * b2 + a1 * b1) * r2 * r2 + 
            ((2.0 * a2 * b3 + 2.0 * a3 * b2) * r1 + 
             (2.0 * a1 * b3 - 2.0 * a3 * b1) * r0) * r2 + 
            ((-a3) * b3 + a2 * b2 + a1 * b1) * r1 * r1 + 
            (2.0 * a1 * b2 - 2.0 * a2 * b1) * r0 * r1 + 
            ((-a3) * b3 - a2 * b2 - a1 * b1) * r0 * r0)

# (2) Define the cost function (here using autograd.numpy)
def cost(X): 
    return np.sum([costk(a, b, X) for a, b in zip(points_a, points_b)])

def cost2(X): 
    return np.sum([costk2(a, b, X) for a, b in zip(points_a, points_b)])

In [184]:
n_points=3

m0 = create_motor()
points_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)).null() for i in range(n_points)]
points_b = [point.spin(m0) for point in points_a]
manifold = Motor()
problem = Problem(manifold=manifold, cost=cost3, grad=grad_explicit3, verbosity=0)


# (3) Instantiate a Pymanopt solver
solver = SteepestDescent()


# let Pymanopt do the rest
Xopt = solver.solve(problem, x=np.array([1.,0.,0.,0.,0.,0.,0.,0.]))
print(vsr.Mot(*Xopt))
print(la.norm(vsr.Mot(*Xopt).rev() * m0))

Mot: [ 0.2 -0.36 -0.73 -0.55 -0.093 -0.032 0.11 -0.059 ]
1.00000771417


In [65]:
manifold = Sphere(4)
problem = Problem(manifold=manifold, cost=cost, verbosity=3)

# (3) Instantiate a Pymanopt solver
solver = SteepestDescent()
# solver = TrustRegions()
# solver = ParticleSwarm()
# solver = ConjugateGradient()
# solver = NelderMead()


# let Pymanopt do the rest
Xopt = solver.solve(problem, x=np.array([1.,0.,0.,0.]))
print(vsr.Rot(*Xopt))
print(vsr.Rot(*Xopt).rev() * m0)

Compiling cost function...
Computing gradient of cost function...
 iter		   cost val	    grad. norm


ValueError: too many values to unpack (expected 3)